**ハミルトン閉路問題**
：「与えられた連結グラフが，すべての頂点をちょうど1回ずつ通る閉路を含むか」すべての頂点を1回ずつ通るが，閉路にならずに道となる(始点と終点が一致しない)場合は，**ハミルトン路**という．

ハミルトン閉路問題を，重みのついた辺によるグラフ考え，このグラフのある閉路がハミルトン閉路になっているとする．このとき閉路のコストは，閉路の辺の重みをすべて足し合わせたものとなる．

**巡回セールスマン問題**
：ハミルトン閉路の中で，コストが最小となるものを探す．

`pow(base, exp[, mod])`
base の exp 乗を返します; mod があれば、base の exp 乗に対する mod の剰余を返します 

In [11]:
def prime_test(n):
    """引数が素数かどうかを判定する"""
    m = int(pow(n, 0.5))  #  ルートnまで調べれば十分
    for d in range(2, m + 1): 
        if n % d == 0:
            return False
    return True

In [8]:
# ｎが大きくなっても正確に計算するためのコード

# 3.7以前ではdecimalを利用する。
from decimal import Decimal, getcontext

def prime_test_large_n(n):
    # 計算の精度を指定する
    getcontext().prec = len(str(n))
    m = int(Decimal(n).sqrt())
    for d in range(2, m + 1):
        if n % d == 0:
            return False
    return True

In [9]:
# 3.8以降では、math.isqrtが利用できる。
import math

def prime_test(n):
    """引数が素数かどうかを判定する"""
    m = math.isqrt(n)
    for d in range(2, m + 1):
        if n % d == 0:
            return False
    return True

In [12]:
prime_test(71)

True

In [13]:
prime_test(5489)

False

In [14]:
prime_test(2147483647)

True

In [ ]:
# これは諦めた方がいい。
prime_test(2305843009213693951)

**空間複雑性** 
：アルゴリズムの実行に際してどれくらいのメモリが必要となるか

ナップザック問題の動的計画法による解放では，品物の数をn, ナップザックの容量をmとすると，途中の計算でn×mの表が必要であった．この表のために確保されるメモリは，品物の数やナップザックの容量に応じて増えることに注意が必要．

一般に計算の複雑性という場合，時間と空間(メモリ)の両方を考える必要がある．

## PとNP
配列のソートやグラフの最短距離を求める問題は，普通のコンピュータを使って多項式時間でとくことができる．このような問題を**多項式**(Polynomial)の頭文字をとって，**クラスP**または単にPと呼ぶ．

これに対し，普通のコンピュータでは多項式時間で解けないが，非決定性チューリング機械を使えば多項式で解ける問題を，**クラスNP**または単にNPと呼ぶ．非決定性チューリング機械とは，計算の途中にある分岐で，すべての道筋を実行できる(=分岐でどれかを選ぶか決める必要がない)コンピュータのことである．

問題がクラスNPに属するためには，**判定問題**である必要がある．判定問題(あるいは決定問題)とは，計算の結果がyesかnoで返ってくる問題．素数判定の問題や，ハミルトン閉路問題は判定問題である．ナップザック問題は，価値を最大化する品物の組み合わせを求める問題であり，最適化問題と呼ばれる．問題を少し変形して，「重量制限があるナップザックに入れる品物の組み合わせで価値がある定数cを超える組み合わせがあるか」という問題を考える．これはyesかnoで答えられるので判定問題である．

## 3SAT
TrueまたはFalseを保持する変数をいくつか並べて，論理和でつなげたもの（節）を，さらに論理積でつなげたものがTrueとなる変数の組み合わせが存在するか，という判定問題．各変数にTrueやFalseを割り当てて計算すれば，全体がTrueとなるかどうか確かめることはできる．3SATの3とは，各節に含まれるリテラルの数が高々3つであることを示している．

kSATクラスを作り，乱数を使って問題を生成するクラスメソッドを追加する．kSATのkと変数の数，節の数を指定できるようにする．

インスタンスメソッドが`self`でインスタンス自身を参照するように、`cls`はメソッド内でクラス自身を参照できるようにするための引数

In [33]:
import random
random.seed(8)

class kSAT:
    
    @classmethod
    def generate(cls, k, var_num, clause_num):
        """変数の数（var_num）と節の数（clause_num）をとりkSAT問題を作る"""
        ksat = cls()
        var_list = list(range(var_num))
        # 問題の本体を格納するための変数
        res = []
        while len(res) < clause_num:
            clause = []
            # 高々k個の変数が含まれる
            clause_size = random.randint(1, k)
            for i in random.sample(var_list, clause_size):
                # 1ならnotで変数を否定する
                prefix = random.choice((0, 1))
                clause.append((prefix, i))
            # 同一の節を判定できるよう変数の添え字でソート
            print("clause : ", clause)
            clause.sort(key=lambda x: x[1])
            print("clause (変数の添字でソート) ", clause)
            if clause not in res: res.append(clause)  # 同一の節がすでに存在すれば，追加しても意味がない
        # ｋSATのインスタンスに格納
        ksat.body = res  # clause(節)が何個か入っている
        return ksat
    
    def test(self, var_list):
        """受け取ったvar_listのTrue、Falseを使って論理式を評価する"""
        res = []
        for clause in self.body:  # 節を一つずつ取り出して，各節の真偽をresに追加していく
            clause_data = [not var_list[i] if p else var_list[i] for p, i in clause]  # prefix(p)が１なら変数を否定する
            # 各節はどれか1つでもTrueならTrue
            print("clause_data : ", clause_data)
            res.append(any(clause_data))
        # 全体は、すべてがTrueならTrue
        return all(res)
    
    def __str__(self):
        res = []
        for clause in self.body:
            clause_str = [f'¬x{i}' if p else f'x{i}' for p, i in clause]
            res.append('(' + ' ∨ '.join(clause_str) + ')')
        return ' ∧ '.join(res)

In [34]:
ksat = kSAT.generate(3, 4, 3)
print(ksat)

clause :  [(1, 2)]
clause (変数の添字でソート)  [(1, 2)]
clause :  [(0, 1)]
clause (変数の添字でソート)  [(0, 1)]
clause :  [(0, 1)]
clause (変数の添字でソート)  [(0, 1)]
clause :  [(1, 1), (1, 3), (1, 0)]
clause (変数の添字でソート)  [(1, 0), (1, 1), (1, 3)]
(¬x2) ∧ (x1) ∧ (¬x0 ∨ ¬x1 ∨ ¬x3)


In [38]:
while True:
    cand = random.choices([True, False], k=4)
    print("cand : ", cand)
    if ksat.test(cand):
        print("find True : ", cand)  # これが出力されれば，全体がTrueになる変数の組み合わせが見つかったということ．
        break

cand :  [True, False, False, False]
clause_data :  [True]
clause_data :  [False]
clause_data :  [False, True, True]
cand :  [True, True, False, True]
clause_data :  [True]
clause_data :  [True]
clause_data :  [False, False, False]
cand :  [False, False, True, True]
clause_data :  [False]
clause_data :  [False]
clause_data :  [True, True, False]
cand :  [True, False, False, True]
clause_data :  [True]
clause_data :  [False]
clause_data :  [False, True, False]
cand :  [True, True, False, False]
clause_data :  [True]
clause_data :  [True]
clause_data :  [False, False, True]
find True :  [True, True, False, False]


# 練習問題解答

## 8.1

指定された桁数の正の整数をランダムに得る関数は、練習問題3.1で実装した。

In [39]:
import random

def rand_n_digit_int(n):
    return random.randint(10**(n-1), 10**n - 1)

1以上の整数を引数にとり、その桁数の素数を探す関数を作る。

In [40]:
def explore_n_digit_prime(n):
    cnt = 1
    while True:
        d = rand_n_digit_int(n)
        if prime_test(d):
            break
        cnt += 1
    return d, cnt

引数を8にすれば8桁の素数を見つけることができる。

In [41]:
d, cnt = explore_n_digit_prime(8)

print(f'{d}を{cnt}回目で見つけた。')

67378279を5回目で見つけた。


$n$が大きくなると、prime_testの実行に時間がかかるようになる。また、10桁を超えるような場合は、関数内部で平方根を整数にする部分で計算誤差の影響がでる可能性がある。これはprime_test_large_nを使うなどの方法で回避できる。計算量の問題も含め、このあたりのややこしい問題をすべて解決してくれる方法を9章で説明する。

## 8.2

ここでは大まかな時間がわかれば良いので、timeモジュールを使う。コードのパフォーマンスを測定するには、通常timeitモジュールを使って実装した方が正確になる。

In [48]:
[random.random() for i in range(rand_n_digit_int(1))]  # 1桁の正の数を乱数として生成し，その数と同じ長さのリストを１つ作る

[0.04945009879317108, 0.6980985166587703, 0.22605684996550823]

In [53]:
import time

def create_n_list(n):
    s = time.time()
    [random.random() for i in range(rand_n_digit_int(n))]
    e = time.time()
    return e - s

In [54]:
for i in range(1, 9):
    print('{}\t{}'.format(i, create_n_list(i)))

1	1.1920928955078125e-05
2	1.71661376953125e-05
3	4.673004150390625e-05
4	0.001215219497680664
5	0.007560014724731445
6	0.11465907096862793
7	0.825659990310669
8	5.1415088176727295


8桁目で時間が7桁の時の約10倍になっているのがわかる。9桁目も10倍の時間で済むならば、それほどかからないようにも思えるが、これにはメモリの問題が影響してくる。次の問題でそれを検討する。

## 8.3

sys.getsizeofは、引数にとったオブジェクトのサイズをbyte単位で返す。大きな数字になるとわかりにくいので、1024で2回割って、MB単位で表示する。

In [56]:
import sys

def create_n_list(n):
    s = time.time()
    test_list = [random.random() for i in range(rand_n_digit_int(n))]
    e = time.time()
    return e - s, sys.getsizeof(test_list)

In [58]:
for i in range(1, 10):
    t, s = create_n_list(i)
    print(f'{i}\t{t}\t{s/1024/1024}')

1	1.5020370483398438e-05	0.00012969970703125
2	1.621246337890625e-05	0.00051116943359375
3	1.9073486328125e-05	0.00102996826171875
4	0.00041174888610839844	0.02550506591796875
5	0.0028276443481445312	0.21497344970703125
6	0.05741095542907715	4.091285705566406
7	0.6605348587036133	61.43299102783203
8	8.94231104850769	819.8971557617188


KeyboardInterrupt: 

8桁（1億から10億未満）にもなる長さでは、サイズが数百MBほどになる。このまま9桁のサイズになると数GBになってしまう。これは一般的なパーソナルコンピュータのメモリサイズと同じくらいだ。かなりの高性能機でなければ、9桁のサイズのリストは扱わない方が良いだろう。

実際には、リストを作ったあとさまざまな計算をすることになる。6桁（数百万）〜7桁（数千万）くらいのサイズを越えるようであれば、データベースサーバの導入など、Pythonプログラムだけで処理しない方法を検討したほうがよいだろう。